# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [5]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [27]:
import pandas as pd
import numpy as np

def Energy_DF():
    Energy = (pd.read_excel('assets/EnergyIndicators.xls', skiprows=17, skipfooter=10)
                .drop(['Unnamed: 0','Unnamed: 1'], axis=1)
                .replace('...',np.NaN))
    Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    Energy['Energy Supply'] = 1000000*Energy['Energy Supply']

    # cleaning up country names before replacing I think makes sense
    Energy['Country']= Energy['Country'].str.replace(r"\(.*\)", "").str.strip()
    Energy['Country']= Energy['Country'].str.replace(r"\d+$", "").str.strip()

    Energy.replace(to_replace=[ "Republic of Korea", 
                                "United States of America", 
                                "United Kingdom of Great Britain and Northern Ireland",
                                "China, Hong Kong Special Administrative Region"]
                   ,value=["South Korea", 
                            "United States",
                            "United Kingdom",
                            "Hong Kong"]
                   ,inplace=True)

    Energy.set_index('Country', inplace=True)
    return Energy

def GDP_DF():
    GDP = pd.read_csv("assets/world_bank.csv",skiprows=4)
    GDP["Country Name"].replace(to_replace = ["Korea, Rep.",
                                              "Iran, Islamic Rep.",
                                              "Hong Kong SAR, China"]
                                ,value=["South Korea", 
                                    "Iran",
                                    "Hong Kong" ]
                                ,inplace = True)

    # Use only the last 10 years (2006-2015) of GDP data
    cols_to_keep=['Country Name', '2006','2007', '2008', '2009', '2010', '2011','2012', '2013', '2014', '2015']
    GDP = GDP[cols_to_keep]
    GDP.set_index('Country Name', inplace=True)
    return GDP

def ScimEn_DF():
    ScimEn = pd.read_excel('assets/scimagojr-3.ods')

    # Use only the  the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).
    ScimEn = ScimEn[ScimEn['Rank']<16]
    ScimEn.set_index('Country', inplace=True)
    return ScimEn

    df = (Energy.merge(ScimEn,how='inner', left_index=True, right_index=True)
                .merge(GDP, how='inner', left_index=True, right_index=True)
                .sort_values('Rank', axis=0))
    
# test
Energy = Energy_DF()
ScimEn = ScimEn_DF()
GDP = GDP_DF()
df_inner = (Energy.merge(ScimEn,how='inner', left_index=True, right_index=True)
                  .merge(GDP, how='inner', left_index=True, right_index=True)
                  .sort_values('Rank', axis=0))
df_inner.shape
inner_rows = df_inner.shape[0]
print(inner_rows)

df_outer = (Energy.merge(ScimEn,how='outer', left_index=True, right_index=True)
                  .merge(GDP, how='outer', left_index=True, right_index=True)
                  .sort_values('Rank', axis=0))
outer_rows = df_outer.shape[0]
print(GDP)
print(outer_rows)
df_inner.head()


15
                          2006          2007          2008          2009  \
Country Name                                                               
Aruba                      NaN           NaN           NaN           NaN   
Andorra           4.018196e+09  4.021331e+09  3.675728e+09  3.535389e+09   
Afghanistan       1.030523e+10  1.172119e+10  1.214448e+10  1.469733e+10   
Angola            5.581103e+10  6.842044e+10  7.787420e+10  7.975320e+10   
Albania           9.771760e+09  1.034829e+10  1.112752e+10  1.150029e+10   
...                        ...           ...           ...           ...   
Yemen, Rep.       2.672565e+10  2.761787e+10  2.872656e+10  2.991436e+10   
South Africa      3.402852e+11  3.585261e+11  3.699668e+11  3.642764e+11   
Congo, Dem. Rep.  1.650894e+10  1.754232e+10  1.863448e+10  1.916651e+10   
Zambia            1.440569e+10  1.560892e+10  1.682234e+10  1.837342e+10   
Zimbabwe          1.006276e+10  9.695130e+09  7.982103e+09  8.459783e+09   

        

,Energy Supply,Energy Supply per Capita,% Renewable,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
China,1.271910e+11,93.0,19.75491,1,127050,126767,597237,411683,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,9.083800e+10,286.0,11.57098,2,96661,94747,792274,265436,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,1.898400e+10,149.0,10.23282,3,30504,30287,223024,61554,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,7.920000e+09,124.0,10.60047,4,20944,20357,206091,37874,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,3.070900e+10,214.0,17.28868,5,18534,18301,34266,12422,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


In [20]:
def answer_one():
    import pandas as pd
    import numpy as np
    
    # YOUR CODE HERE
    Energy = Energy_DF()
    ScimEn = ScimEn_DF()
    GDP = GDP_DF()
    df_inner = (Energy.merge(ScimEn,how='inner', left_index=True, right_index=True)
                      .merge(GDP, how='inner', left_index=True, right_index=True)
                      .sort_values('Rank', axis=0))
    return df_inner
    
    raise NotImplementedError()

In [21]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [22]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [23]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [58]:
def get_ScimEnFull():
    # returne the whole ScimEn data set rather than the Top 15 ranked
    ScimEnFull = pd.read_excel('assets/scimagojr-3.ods')
    ScimEnFull.set_index('Country', inplace=True)
    return ScimEnFull
    
def get_df_outer():
    # need to use the whole ScimEn data frame
    ScimEnFull = get_ScimEnFull()
    df_outer = (Energy.merge(ScimEnFull,how='outer', left_index=True, right_index=True)
                .merge(GDP, how='outer', left_index=True, right_index=True)
                .sort_values('Rank', axis=0))
    return (df_outer)



In [61]:
def answer_two():
    # YOUR CODE HERE
    inner_rows = answer_one().shape[0]
    df_outer = get_df_outer()
#     df_outer = (Energy.merge(ScimEn,how='outer', left_index=True, right_index=True)
#                   .merge(GDP, how='outer', left_index=True, right_index=True)
#                   .sort_values('Rank', axis=0))
    outer_rows = df_outer.shape[0] 
    return outer_rows-inner_rows
    
    raise NotImplementedError()
print (answer_two())

332


In [25]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


In [54]:
inner_rows = answer_one().shape[0]
print (inner_rows)


15


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def answer_three():
    # YOUR CODE HERE
    df_outer = get_df_outer()
    Energy = Energy_DF()
    ScimEnFull = get_ScimEnFull()
    GDP = GDP_DF()

    # Need inner join with all ScimEN records included
    df_all_inner = (Energy.merge(ScimEnFull,how='inner', left_index=True, right_index=True)
                    .merge(GDP, how='inner', left_index=True, right_index=True)
                    .sort_values('Rank', axis=0))

    cols = df_all_inner.loc[:,'2006':'2015']
    df_all_inner['mean'] = cols.mean(axis=1)
    df_largest_gdp = df_all_inner.nlargest(15, 'mean')

    avgGDP = df_largest_gdp[df_largest_gdp.columns[-1]]
    return avgGDP
    raise NotImplementedError()

In [ ]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
def answer_four():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [ ]:
def answer_five():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [ ]:
def answer_eight():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [ ]:
def answer_thirteen():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")